# Import Bibliotheken

In [1]:
# Festlegung des Device
import platform

# Laden der Daten
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

# Operationen mit Texten
import torchtext
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import spacy

# Modell
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

# Evaluierung
import torchmetrics

# Auswertung der Evaluation
import pandas as pd
import altair as alt
import numpy as np

from sklearn import preprocessing

### Package Versionen

In [2]:
for package in [torchtext,torch, torchmetrics, spacy, alt, pd]:
    print(f'{package.__name__}: {package.__version__}')

torchtext: 0.13.1
torch: 1.12.1
torchmetrics: 0.9.3
spacy: 3.4.2
altair: 4.2.0
pandas: 1.4.3


# Device Auswahl

In [3]:
device = torch.device(
    "cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

# Daten

## Dataset

### Erstellung Dataset

In [4]:
comment_df = pd.read_excel('../data/clean/Google_Rezensionen.xlsx')
comment_df

,Unnamed: 0,Tankstellenname,Datum,Bewertung,Kommentar,Kategorien
0,4,TS Schleswig MC,2022-09-30 06:46:42,5,super nettes personal gutes frühstück mit groß...,"Personal,Shop,Bistro"
1,5,TS Prisdorf,2022-09-29 17:22:44,5,sehr höfliche bedienstete,Personal
2,6,TS Wanderup,2022-09-29 15:38:02,5,immer gerne dort nettes und zuvorkommendes per...,"Personal,Waschanlage"
3,10,TS Handewitt,2022-09-28 17:05:28,5,nettes freundliches personal,Personal
4,12,TS Bremen,2022-09-28 04:24:34,4,normale tankstelle die aber in der tank app ni...,"Pricing,DigitalFueling"
...,...,...,...,...,...,...
3992,11072,TS Jübek,2015-04-07 17:45:02,5,dies ist einfach die beste tankstelle die ich ...,"Kraftstoffauswahl,Waschanlage"
3993,11077,TS Jübek,2014-07-16 07:08:37,5,sehr gut günstig guter shop und einige andere ...,"Pricing,Shop,Kraftstoffauswahl,Waschanlage,Sta..."
3994,11079,TS Neustadt am Rüb.,2013-09-21 15:26:28,5,öffnungszeiten mo so 06 00 22 00 uhr,Öffnungszeiten
3995,11080,TS Handewitt(SP),2012-11-05 11:16:00,5,genug platz fuer wohnwagen rechts beim autogas...,"Personal,Kraftstoffauswahl"


In [5]:
class TeamGoogleBewertungenDataSet(Dataset):

    def __init__(self, data):
        self.data = data
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        text = comment_df.iloc[index]['Kommentar']
        category = comment_df.iloc[index]['Kategorien']
        return text, category

### Bestimmung Test- und Traningsdaten

In [6]:
# Bestimmung der Länge der Test- und Traningsdaten
dataset = TeamGoogleBewertungenDataSet(comment_df)
len_train = round(len(dataset)* 0.7)
len_test = round(len(dataset) * 0.3)
assert len(comment_df) == len_train + len_test

In [7]:
# Zufällige Aufsplittung der Anzahl von Test- und Trainingsdaten
train_set, test_set = torch.utils.data.random_split(dataset, [len_train, len_test]) 

In [8]:
# next(iter(train_set))

## Dataloader

### Tokenisierung

In [9]:
tokenizer = get_tokenizer(
    'spacy', 
    language='de_core_news_lg')

In [10]:
# Beispiel für Anwendung des tokenizer
# sentence = "This isn't a very long example."
# tokenizer(sentence)

### Erstellung Vokabular

In [11]:
def yield_tokens(data_iter):
    for text, _ in data_iter:
        yield tokenizer(text)

In [12]:
# Länge des Trainungsets
len(train_set)

2798

In [13]:
tokens = yield_tokens(train_set)

In [14]:
next(iter(tokens))

['letzte',
 'tankstelle',
 'vor',
 'der',
 'grenze',
 'nähe',
 'an',
 'der',
 'autobahn']

In [15]:
vocab = build_vocab_from_iterator(tokens, min_freq=2, specials=["<unk>"])

vocab.set_default_index(vocab["<unk>"])

In [16]:
len(vocab)

2372

In [17]:
vocab(['Hallo', 'sehr', 'Personal'])

[0, 3, 0]

In [18]:
vocab.lookup_token(0)

'<unk>'

### Encoding

In [19]:
onehot = torch.zeros(1,len(vocab))

In [20]:
pos = vocab(['nette', 'Bedienung', 'sauber'])
pos

[35, 0, 43]

In [21]:
def encode(text, vocab):
    tokens = tokenizer(text)
    onehot = torch.zeros(1,len(vocab))
    onehot[:,vocab(tokens)] = 1
    return onehot

### Multilabel Encoding

In [22]:
import torch
import torch.nn

In [23]:
df = comment_df.explode('Kategorien')
df

,Unnamed: 0,Tankstellenname,Datum,Bewertung,Kommentar,Kategorien
0,4,TS Schleswig MC,2022-09-30 06:46:42,5,super nettes personal gutes frühstück mit groß...,"Personal,Shop,Bistro"
1,5,TS Prisdorf,2022-09-29 17:22:44,5,sehr höfliche bedienstete,Personal
2,6,TS Wanderup,2022-09-29 15:38:02,5,immer gerne dort nettes und zuvorkommendes per...,"Personal,Waschanlage"
3,10,TS Handewitt,2022-09-28 17:05:28,5,nettes freundliches personal,Personal
4,12,TS Bremen,2022-09-28 04:24:34,4,normale tankstelle die aber in der tank app ni...,"Pricing,DigitalFueling"
...,...,...,...,...,...,...
3992,11072,TS Jübek,2015-04-07 17:45:02,5,dies ist einfach die beste tankstelle die ich ...,"Kraftstoffauswahl,Waschanlage"
3993,11077,TS Jübek,2014-07-16 07:08:37,5,sehr gut günstig guter shop und einige andere ...,"Pricing,Shop,Kraftstoffauswahl,Waschanlage,Sta..."
3994,11079,TS Neustadt am Rüb.,2013-09-21 15:26:28,5,öffnungszeiten mo so 06 00 22 00 uhr,Öffnungszeiten
3995,11080,TS Handewitt(SP),2012-11-05 11:16:00,5,genug platz fuer wohnwagen rechts beim autogas...,"Personal,Kraftstoffauswahl"


#### Kategorien splitten

In [24]:
einzelLabels = comment_df['Kategorien'].str.split(',')
label_list_export = einzelLabels.to_list()
label_list_export

[['Personal', 'Shop', 'Bistro'],
 ['Personal'],
 ['Personal', 'Waschanlage'],
 ['Personal'],
 ['Pricing', 'DigitalFueling'],
 ['Erscheinungsbild', 'Personal'],
 ['Personal', 'Pricing'],
 ['Personal', 'Pricing', 'Kraftstoffauswahl'],
 ['Personal', 'Öffnungszeiten'],
 ['Personal', 'SB-Waschboxen', 'Waschanlage'],
 ['Shop'],
 ['Erscheinungsbild', 'Personal'],
 ['Personal', 'Waschanlage'],
 ['Personal', 'Pricing'],
 ['Personal', 'Pricing'],
 ['Bistro'],
 ['Personal', 'Pricing', 'Shop', 'Waschanlage'],
 ['Personal'],
 ['Personal'],
 ['Bistro'],
 ['Erscheinungsbild'],
 ['Personal', 'Pricing'],
 ['Personal'],
 ['Personal'],
 ['Personal', 'Shop'],
 ['Erscheinungsbild'],
 ['Personal', 'Bistro'],
 ['Erscheinungsbild', 'Nacht-/Tankautomat', 'Waschanlage', 'Staubsauger'],
 ['Pricing'],
 ['Personal'],
 ['Erscheinungsbild', 'Personal'],
 ['Personal'],
 ['Bistro'],
 ['Pricing', 'Kraftstoffauswahl'],
 ['Pricing'],
 ['Nacht-/Tankautomat'],
 ['Shop', 'Bistro'],
 ['Erscheinungsbild', 'Personal', 'Sanitär

##### In der nachfolgenden Zelle wird aus der verschachtelten Liste eine "flache Liste"

In [25]:
flat_list = []
for sublist in label_list_export:
    for item in sublist:
        flat_list.append(item)

flat_list

['Personal',
 'Shop',
 'Bistro',
 'Personal',
 'Personal',
 'Waschanlage',
 'Personal',
 'Pricing',
 'DigitalFueling',
 'Erscheinungsbild',
 'Personal',
 'Personal',
 'Pricing',
 'Personal',
 'Pricing',
 'Kraftstoffauswahl',
 'Personal',
 'Öffnungszeiten',
 'Personal',
 'SB-Waschboxen',
 'Waschanlage',
 'Shop',
 'Erscheinungsbild',
 'Personal',
 'Personal',
 'Waschanlage',
 'Personal',
 'Pricing',
 'Personal',
 'Pricing',
 'Bistro',
 'Personal',
 'Pricing',
 'Shop',
 'Waschanlage',
 'Personal',
 'Personal',
 'Bistro',
 'Erscheinungsbild',
 'Personal',
 'Pricing',
 'Personal',
 'Personal',
 'Personal',
 'Shop',
 'Erscheinungsbild',
 'Personal',
 'Bistro',
 'Erscheinungsbild',
 'Nacht-/Tankautomat',
 'Waschanlage',
 'Staubsauger',
 'Pricing',
 'Personal',
 'Erscheinungsbild',
 'Personal',
 'Personal',
 'Bistro',
 'Pricing',
 'Kraftstoffauswahl',
 'Pricing',
 'Nacht-/Tankautomat',
 'Shop',
 'Bistro',
 'Erscheinungsbild',
 'Personal',
 'Sanitär',
 'Bistro',
 'Pricing',
 'Verkehrsanbindung'

##### Doppelte Werte werden entfernt, so dass jeder Eintrag nur einmalig vorhanden ist. Dies erreichen wir mit set()

In [26]:
flat_set = set(flat_list)
flat_set

{'AdBlue',
 'Bistro',
 'DigitalFueling',
 'E-Fuels',
 'E-Mobilität',
 'Erscheinungsbild',
 'Kartenakzeptanzen',
 'Kraftstoffauswahl',
 'Luft',
 'Nacht-/Tankautomat',
 'Paketservice',
 'Personal',
 'Pricing',
 'SB-Waschboxen',
 'Sanitär',
 'Shop',
 'Staubsauger',
 'Tankpool',
 'Verkehrsanbindung',
 'WLAN',
 'Waschanlage',
 'Werkstatt',
 'Zapfsäulen',
 'Öffnungszeiten'}

In [27]:
unflat = [[x] for x in flat_set]

In [28]:
label_vocab = build_vocab_from_iterator(unflat, min_freq=1,)

label_vocab.set_default_index(vocab["<unk>"])

In [29]:
label_vocab.lookup_token(3)

'E-Fuels'

#### Dictionary erstellen

In [30]:
label_dict = {label:i for i, label in enumerate(flat_set)}
label_dict

{'Öffnungszeiten': 0,
 'Personal': 1,
 'Tankpool': 2,
 'Paketservice': 3,
 'SB-Waschboxen': 4,
 'Staubsauger': 5,
 'AdBlue': 6,
 'Werkstatt': 7,
 'Waschanlage': 8,
 'Verkehrsanbindung': 9,
 'Kartenakzeptanzen': 10,
 'Zapfsäulen': 11,
 'Nacht-/Tankautomat': 12,
 'Bistro': 13,
 'Luft': 14,
 'E-Mobilität': 15,
 'Shop': 16,
 'Pricing': 17,
 'Erscheinungsbild': 18,
 'E-Fuels': 19,
 'DigitalFueling': 20,
 'Kraftstoffauswahl': 21,
 'WLAN': 22,
 'Sanitär': 23}

#### Tensor

In [31]:
doc2 = label_list_export[2]
doc2

['Personal', 'Waschanlage']

In [32]:
pos_vals = [label_dict[val] for val in doc2]
pos_vals

[1, 8]

In [33]:
labels = torch.LongTensor(pos_vals)
labels

tensor([1, 8])

In [34]:
y_onehot = nn.functional.one_hot(labels, num_classes=24)
y_onehot

tensor([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [35]:
y_onehot = y_onehot.sum(dim=0).float()
y_onehot

tensor([0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.])

In [36]:
# labels = torch.tensor([1,5,8,0])
# labels = labels.unsqueeze(0)
# target = torch.zeros(labels.size(0), 24).scatter_(1, labels, 1.)
# print(target)

### Erstellung Dataloader

In [37]:
# label_pipeline = lambda x: int(x) -1

In [38]:
def collate_batch(batch):
    label_list, text_list = [], []
 
    for (_text,_label) in batch:
    
        # Vorverarbeitung der Label
        # label_list.append(label_pipeline(_label))
        processed_labels = encode(_label, label_vocab)

        label_list.append(processed_labels)

        # Vorverarbeitung der Texte
        processed_text = encode(_text, vocab)
        
        # Zusammenführen sämtlicher Textrepräsentationen in einer Liste
        text_list.append(processed_text)
 
    # Zusammenführen aller Label in einem Tensor
    labels = torch.cat(label_list, dim=0)
    
    # Verbinden der Tensoren in text_list zu einem Tensor
    texts = torch.cat(text_list, dim = 0)

    # Ausgabe der Texte und der Label
    return texts.to(device), labels.to(device)

In [39]:
train_loader = DataLoader(
 train_set, batch_size=64,
 shuffle=True,
 collate_fn=collate_batch,
 num_workers=0
)

## Architektur und Training

### Architektur

In [40]:
class LinearTextClassificationModel(nn.Module):

    def __init__(self, vocab_size, num_class):
        super(LinearTextClassificationModel, self).__init__()
        self.fc1 = nn.Linear(vocab_size, 200)
        self.fc2 = nn.Linear(200, num_class)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        output = F.softmax(x, dim=1)
        return output

In [41]:
vocab_size = len(vocab)
num_class = 24
model = LinearTextClassificationModel(vocab_size, num_class).to(device)

### Training

In [42]:
# Hyperparameter

## Festlegung Lernrate
learning_rate = 0.03

## Initialisierung Fehlerfunktion
loss_fn = nn.CrossEntropyLoss()

## Initialisierung Fehlerfunktion
bce_loss = nn.BCEWithLogitsLoss()

## Initialisierung Optimizer
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)

## Definition der Epochen
num_epochs = 50

In [43]:
train_accuracy = torchmetrics.Accuracy(subset_accuracy=True, threshold= 0.6).to(device)

loss_hist = {}
accuracy_hist = {}

In [44]:
for (text, label) in train_loader:
    #print(label[[0][0]])
    #print(text)
    labelnew = label.type(torch.LongTensor)
    print(labelnew[[0][0]])

tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0])
tensor([1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0])
tensor([1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0])
tensor([1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([1, 1, 0, 0, 0, 0, 0,

In [45]:
# Training
model.train()

for epoch in range(num_epochs):

    # Dokumentation Loss -> Erkennung ob Netz konvergiert
    running_loss = 0.0
    num_batches = 0

    for (text, label) in train_loader:
        num_batches += 1
        pred = model(text)

        # Der Fehler wird berechnet
        loss = bce_loss(pred, label)
        # Der Fehler wird über das Netz zurückpropagiert
        loss.backward()
        # Die Gewichte werden angepasst
        optimizer.step()
        # Gradienten zurücksetzen
        optimizer.zero_grad()
        ## Bestimmung der Accuracy für den Batch
        train_accuracy(pred, label.type(torch.LongTensor))
        
        # running loss
        running_loss +=loss.item()
 
    loss_hist[epoch] = running_loss/num_batches
 
    batch_train_accuracy = train_accuracy.compute()
    print(f"Training Accuracy for epoch {epoch}:{batch_train_accuracy}")
    accuracy_hist[epoch] = batch_train_accuracy.cpu().item()
    train_accuracy.reset()

Training Accuracy for epoch 0:0.2859185039997101
Training Accuracy for epoch 1:0.3213009238243103
Training Accuracy for epoch 2:0.36347392201423645
Training Accuracy for epoch 3:0.38634738326072693
Training Accuracy for epoch 4:0.39027875661849976
Training Accuracy for epoch 5:0.3974267244338989
Training Accuracy for epoch 6:0.40064331889152527
Training Accuracy for epoch 7:0.39421015977859497
Training Accuracy for epoch 8:0.400285929441452
Training Accuracy for epoch 9:0.4017155170440674
Training Accuracy for epoch 10:0.4074338674545288
Training Accuracy for epoch 11:0.4177984297275543
Training Accuracy for epoch 12:0.4231593906879425
Training Accuracy for epoch 13:0.4238741993904114
Training Accuracy for epoch 14:0.4249463975429535
Training Accuracy for epoch 15:0.42637598514556885
Training Accuracy for epoch 16:0.4267333745956421
Training Accuracy for epoch 17:0.42351680994033813
Training Accuracy for epoch 18:0.4238741993904114
Training Accuracy for epoch 19:0.42637598514556885
Tra

In [46]:
batch_train_accuracy

tensor(0.4303)

### Auswertung Fehlerentwicklung

In [47]:
loss_df = pd.DataFrame.from_dict(loss_hist, orient= 'index').reset_index()

loss_df.columns = ['Epoch', 'Loss']
loss_df.head()

,Epoch,Loss
0,0,0.688573
1,1,0.685965
2,2,0.684252
3,3,0.683032
4,4,0.682921


In [48]:
train_chart = alt.Chart(loss_df).mark_line().encode(
    x=alt.X('Epoch', title = 'Anzahl Epochen'),
    y=alt.Y('Loss', title = 'Mittlerer Fehler')
).properties(title='Trainingsfehlerentwicklung in Abhängigkeit von der Anzahl der Epochen')
train_chart

alt.Chart(...)

### Auswertung Genauigkeit

In [49]:
accuracy_df = pd.DataFrame.from_dict(accuracy_hist, orient = 'index').reset_index()
accuracy_df.columns = ['Epoch', 'Accuracy']
accuracy_df.head()

,Epoch,Accuracy
0,0,0.285919
1,1,0.321301
2,2,0.363474
3,3,0.386347
4,4,0.390279


In [50]:
accuracy_chart = alt.Chart(accuracy_df).mark_line().encode(
    x=alt.X('Epoch',title = 'Anzahl Epochen'),
    y=alt.Y('Accuracy', title = 'Genauigkeit')
).properties(title='Trainingsgenauigkeit in Abhängigkeit von der Anzahl der Epochen')
accuracy_chart

alt.Chart(...)

## Evaluierung

In [51]:
model.eval()

LinearTextClassificationModel(
  (fc1): Linear(in_features=2372, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=24, bias=True)
)

### Gesamtevaluierung

In [52]:
test_loader = DataLoader(test_set, batch_size= 64, shuffle=True,collate_fn=collate_batch)

In [53]:
valid_accuracy = torchmetrics.Accuracy(subset_accuracy=True, threshold=0.6).to(device)

with torch.no_grad():
    for (text, label) in test_loader:
        # Vorhersage wird für das Model erzeugt
        pred = model(text)
        valid_accuracy.update(pred, label.type(torch.LongTensor))
    total_valid_accuracy = valid_accuracy.compute()

In [54]:
total_valid_accuracy

tensor(0.3828)

### Evaluierung nach Klassen

In [55]:
# Evaluieren
valid_accuracy = torchmetrics.Accuracy(
    average=None,
    num_classes=24
).to(device)

with torch.no_grad():
    for(text, label) in test_loader:
        # Die Vorhersage wird für das Model erzeugt
        pred = model(text)
        valid_accuracy.update(pred, label.type(torch.LongTensor))
    total_valid_accuracy = valid_accuracy.compute()
    total_valid_accuracy

In [56]:
total_valid_accuracy

tensor([0.5696, 0.8390, 0.9900,    nan, 0.9983, 0.8724, 0.9633, 0.9316, 0.9833,
        0.9775, 0.9942, 0.8490, 0.8282, 0.9875, 0.9666, 0.8574, 0.9908, 0.9983,
        0.9641,    nan, 0.8724, 0.9900, 0.9858, 0.9633])

### Evaluierung nach der Anzahl der Epochen

In [57]:
train_loader = DataLoader(train_set, batch_size=64, shuffle=True, collate_fn=collate_batch)

In [58]:
test_loader = DataLoader(train_set, batch_size=64, shuffle=True, collate_fn=collate_batch)

In [59]:
vocab_size = len(vocab)
num_class = 24
model = LinearTextClassificationModel(vocab_size, num_class).to(device)

In [60]:
bce_loss = nn.BCEWithLogitsLoss()

In [61]:
learning_rate = 0.03

optimizer = optim.Adam(
    model.parameters(),
    lr=learning_rate,
)

In [62]:
num_epochs = 50

In [63]:
# Evaluierung wurd vorbereitet
train_accuracy = torchmetrics.Accuracy(subset_accuracy=True, threshold=0.6).to(device)
valid_accuracy = torchmetrics.Accuracy(subset_accuracy=True, threshold=0.6).to(device)

train_accuracy_hist = {}
train_loss_hist = {}

valid_accuracy_hist = {}
valid_loss_hist = {}

In [64]:
for epoch in range(num_epochs):

    # Loss wird dokumentiert, sodass zu erkenne ist, ob das Netz konvergiert
    num_train_batches = 0
    num_eval_batches = 0
    train_running_loss = 0.0
    eval_running_loss = 0.0

    model.train()
    for (text, label) in train_loader:
        num_train_batches += 1
        # Die Vorhersage wird für das Model erzeugt
        pred = model(text)
        # Der Fehler wird berechnet
        # Als Ausgabe wird ein Tensor zurückgegeben
        loss = bce_loss(pred, label)
        # Der Fehler wird über das Netz zurükpropagiert
        loss.backward()
        # Die Gewichte werden angepasst
        optimizer.step()
        # Gradienten zurücksetzen
        optimizer.zero_grad()
        # Aktualisierung des States der train_accurracy
        train_accuracy.update(pred, label.type(torch.LongTensor))
        # running loss
        train_running_loss += loss.item()

    train_loss_hist[epoch] = train_running_loss / num_train_batches

    # Berechnung der Trainingsgenauigkeit für die Epoche
    total_train_accuracy = train_accuracy.compute()
    # Hinzufügen der Trainingsgenauigkeit zu dem Dictionary
    train_accuracy_hist[epoch] = total_valid_accuracy.cpu().numpy()

    # Validierung
    model.eval()
    with torch.no_grad():
        for (text, label) in test_loader:
            num_eval_batches += 1
            # Die Vorhersage wird für das Model erzeugt
            pred = model(text)
            # Test loss
            eval_loss = bce_loss(pred, label)
            # Running loss
            eval_running_loss += eval_loss.item()
            # Aktualisierung des States der valid_accuracy
            valid_accuracy.update(pred, label.type(torch.LongTensor))
    
    valid_loss_hist[epoch] = eval_running_loss/num_eval_batches

    # Berechnung der Validierungsgenauigkeit für die Epoche
    total_valid_accuracy = valid_accuracy.compute()
    # Hinzufügen der Validierungsgenauigkeit zum Dictionary
    valid_accuracy_hist[epoch] = total_valid_accuracy.cpu().numpy()

    print(f"Training Accuracy for epoch {epoch}: {total_train_accuracy}")
    print(f"Validation Accuracy for epoch {epoch}: {total_valid_accuracy}")
    train_accuracy.reset()
    valid_accuracy.reset()



Training Accuracy for epoch 0: 0.30307361483573914
Validation Accuracy for epoch 0: 0.3452466130256653
Training Accuracy for epoch 1: 0.35704073309898376
Validation Accuracy for epoch 1: 0.37741243839263916
Training Accuracy for epoch 2: 0.37741243839263916
Validation Accuracy for epoch 2: 0.37562546133995056
Training Accuracy for epoch 3: 0.3795568346977234
Validation Accuracy for epoch 3: 0.39492493867874146
Training Accuracy for epoch 4: 0.39313796162605286
Validation Accuracy for epoch 4: 0.4010007083415985
Training Accuracy for epoch 5: 0.40064331889152527
Validation Accuracy for epoch 5: 0.4017155170440674
Training Accuracy for epoch 6: 0.39957112073898315
Validation Accuracy for epoch 6: 0.4027877151966095
Training Accuracy for epoch 7: 0.4045746922492981
Validation Accuracy for epoch 7: 0.4063616991043091
Training Accuracy for epoch 8: 0.4010007083415985
Validation Accuracy for epoch 8: 0.40600427985191345
Training Accuracy for epoch 9: 0.4067190885543823
Validation Accuracy fo

In [65]:
train_loss_df = pd.DataFrame.from_dict(train_loss_hist, orient = "index").reset_index()
train_loss_df["type"] = "train"
valid_loss_df = pd.DataFrame.from_dict(valid_loss_hist, orient = "index").reset_index()
valid_loss_df["type"] = "validation"
loss_df = pd.concat([train_loss_df,valid_loss_df])
loss_df.columns = ["Epoch", "Loss", "Type"]

In [66]:
train_eval_loss_chart = alt.Chart(loss_df).mark_line().encode(
 x='Epoch',
 y='Loss',
 color='Type'
).properties(title='Fehlerentwicklung in Abhängigkeit von der Anzahl der Epochen')
train_eval_loss_chart

alt.Chart(...)

In [67]:
valid_accuracy_hist

{0: array(0.3452466, dtype=float32),
 1: array(0.37741244, dtype=float32),
 2: array(0.37562546, dtype=float32),
 3: array(0.39492494, dtype=float32),
 4: array(0.4010007, dtype=float32),
 5: array(0.40171552, dtype=float32),
 6: array(0.40278772, dtype=float32),
 7: array(0.4063617, dtype=float32),
 8: array(0.40600428, dtype=float32),
 9: array(0.40743387, dtype=float32),
 10: array(0.37776983, dtype=float32),
 11: array(0.40814868, dtype=float32),
 12: array(0.40957826, dtype=float32),
 13: array(0.41065046, dtype=float32),
 14: array(0.41065046, dtype=float32),
 15: array(0.41136527, dtype=float32),
 16: array(0.41172266, dtype=float32),
 17: array(0.41172266, dtype=float32),
 18: array(0.41208005, dtype=float32),
 19: array(0.41136527, dtype=float32),
 20: array(0.41243747, dtype=float32),
 21: array(0.41208005, dtype=float32),
 22: array(0.41279486, dtype=float32),
 23: array(0.41243747, dtype=float32),
 24: array(0.41279486, dtype=float32),
 25: array(0.41279486, dtype=float32),

In [68]:
train_accuracy_hist

{0: array([0.56964135, 0.83903253, 0.98999166,        nan, 0.99833196,
        0.87239367, 0.96330273, 0.9316097 , 0.98331946, 0.97748125,
        0.9941618 , 0.84904087, 0.82819015, 0.9874896 , 0.96663886,
        0.85738117, 0.9908257 , 0.99833196, 0.9641368 ,        nan,
        0.87239367, 0.98999166, 0.98582155, 0.96330273], dtype=float32),
 1: array(0.3452466, dtype=float32),
 2: array(0.37741244, dtype=float32),
 3: array(0.37562546, dtype=float32),
 4: array(0.39492494, dtype=float32),
 5: array(0.4010007, dtype=float32),
 6: array(0.40171552, dtype=float32),
 7: array(0.40278772, dtype=float32),
 8: array(0.4063617, dtype=float32),
 9: array(0.40600428, dtype=float32),
 10: array(0.40743387, dtype=float32),
 11: array(0.37776983, dtype=float32),
 12: array(0.40814868, dtype=float32),
 13: array(0.40957826, dtype=float32),
 14: array(0.41065046, dtype=float32),
 15: array(0.41065046, dtype=float32),
 16: array(0.41136527, dtype=float32),
 17: array(0.41172266, dtype=float32),
 

In [69]:
# Erste Epoche löschen, da diese nicht neu initialisiert wird und Daten aus vorherigen Schritten enthält

del train_accuracy_hist[0]
del valid_accuracy_hist[0]

In [70]:
train_accuracy_df = pd.DataFrame.from_dict(train_accuracy_hist, orient = "index").reset_index()
train_accuracy_df["type"] = "train"
valid_accuracy_df = pd.DataFrame.from_dict(valid_accuracy_hist, orient = "index").reset_index()
valid_accuracy_df["type"] = "validation"
accuracy_df = pd.concat([train_accuracy_df,valid_accuracy_df])
accuracy_df.columns = ["Epoch", "Accuracy", "Type"]

In [71]:
accuracy_df

,Epoch,Accuracy,Type
0,1,0.345247,train
1,2,0.377412,train
2,3,0.375625,train
3,4,0.394925,train
4,5,0.401001,train
...,...,...,...
44,45,0.402788,validation
45,46,0.407791,validation
46,47,0.408506,validation
47,48,0.409578,validation


In [72]:
train_eval_accuracy_chart = alt.Chart(accuracy_df).mark_line().encode(
 x='Epoch',
 y='Accuracy',
 color='Type'
).properties(title='Genauigkeiten des Trainings- und Testdatensatzes nach der Anzahl der Epochen')
train_eval_accuracy_chart

alt.Chart(...)